# Filtering Microcap Alts

In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep


# Functions for pulling from Coinmarketcap
- coindfpull: pulls coin information from cmc including total coin supply
- tokendfpull: pulls token information from cmc including total token supply
- coincircdf: pulls coin circulating supply from cmc
- tokencircdf: pulls token circulating supply from cmc

In [2]:
def coindfpull(pagenum):
    site = 'https://coinmarketcap.com/coins/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[9].split('data-btc="')[1].split(' " data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[9].split('data-btc="')[1].split(' " data-usd="')[0]]
            pricesusd = pricesusd + [columns[15].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            pricesbtc = pricesbtc + [columns[15].split('" data-btc="')[1].split('" data-usd="')[0]]
            volumes24husd = volumes24husd + [columns[18].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            volumes24hbtc = volumes24hbtc + [columns[18].split('" data-btc="')[1].split('" data-usd="')[0]]
            totalsupplies = totalsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def tokendfpull(pagenum):
    site = 'https://coinmarketcap.com/tokens/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    platforms = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            platforms = platforms + [columns[0].split('data-platformsymbol="')[1].split('" ')[0]]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[10].split('data-btc="')[1].split(' " ')[1].split('data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[10].split('data-btc="')[1].split(' " ')[0]]
            pricesusd = pricesusd + [columns[16].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            pricesbtc = pricesbtc + [columns[16].split('data-btc="')[1].split('" ')[0]]
            volumes24husd = volumes24husd + [columns[19].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            volumes24hbtc = volumes24hbtc + [columns[19].split('data-btc="')[1].split('" ')[0]]
            totalsupplies = totalsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0].split('" ')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "Platform": platforms,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def coincircdf(pagenum):
    site = 'https://coinmarketcap.com/coins/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

def tokencirdf(pagenum):
    site = 'https://coinmarketcap.com/tokens/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

# Alternative Function for Pulling all Coin/Token Caps

In [17]:
def pullfulldf():
    site = 'https://coinmarketcap.com/all/views/all/'
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")

    table_start = '<table class="table js-summary-table dataTable no-footer" id="currencies-all" style="font-size: 14px; width: 100%;" role="grid">'
    table_end = '</table>'

    table = soup.find_all('table')[0]

    symbol = []
    cname = []
    mcapsusd = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    totalsupplies = []
    availablesupplies = []
    maxsupplies = []
    change1h = []
    change24h = []
    change7d = []
    i = 1
    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if not('class' in columns[0]):
            continue

        cmcname = columns[0].split('"id-')[1].replace('">','')

        apisite = 'https://api.coinmarketcap.com/v1/ticker/' + cmcname + '/'
        response = requests.get(apisite)
        json_data = json.loads(response.text)

        cname = cname + [cmcname]
        symbol = symbol + [json_data[0]['symbol']]
        mcapsusd = mcapsusd + [json_data[0]['market_cap_usd']]
        pricesusd = pricesusd + [json_data[0]['price_usd']]
        pricesbtc = pricesbtc + [json_data[0]['price_btc']]
        volumes24husd = volumes24husd + [json_data[0]['24h_volume_usd']]
        totalsupplies = totalsupplies + [json_data[0]['total_supply']]
        availablesupplies = availablesupplies + [json_data[0]['available_supply']]
        maxsupplies = maxsupplies + [json_data[0]['max_supply']]
        change1h = change1h + [json_data[0]['percent_change_1h']]
        change24h = change24h + [json_data[0]['percent_change_24h']]
        change7d = change7d + [json_data[0]['percent_change_7d']]
        
        print('Number: ' + str(i) + ' Coin: ' + json_data[0]['symbol'])
        sleep(1)
        i += 1

    fulldict = {'CMCname': cname,
                'Symbol': symbol,
                'Mcap': mcapsusd,
                'USDPrice': pricesusd,
                'BTCPrice': pricesbtc,           
                'Vol24h': volumes24husd,           
                'TotalSupply': totalsupplies,           
                'AvailableSupply': availablesupplies,    
                'MaxSupply': maxsupplies,
                'Change1h': change1h,
                'Change24h': change24h,
                'Change7d': change7d}
    fulldf = pd.DataFrame(fulldict,index = cname)
    
    return fulldf


# Creating dataframe for initial filtering
- Keep only marketcaps with less than 100M
- Remove tokens with 24 hour volumes less than 2% of marketcaps
- Remove tokens with circulating supply/total supply with less than 20%

In [10]:
# Creating DF for Coins

def microfilter1(mcapthreshold, volthreshold, supplythreshold):
    pagenmbr = 1
    coinlist = []
    while type(coindfpull(pagenmbr)) != str:
        coinlist = coinlist + [coindfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    coincirclist = []
    while type(coincircdf(pagenmbr)) != str:
        coincirclist = coincirclist + [coincircdf(pagenmbr)]
        pagenmbr += 1

    coinscirc = pd.concat(coincirclist, axis = 0)
    coins = pd.concat(coinlist, axis = 0)
    coins = coins[coins['MktCapUSD'] != '?']
    coins = pd.concat([coins.drop_duplicates(),coinscirc.drop_duplicates()], axis = 1, join = 'inner')

    # Creating DF for Tokens

    pagenmbr = 1
    tokenlist = []
    while type(tokendfpull(pagenmbr)) != str:
        tokenlist = tokenlist + [tokendfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    tokencirclist = []
    while type(tokencirdf(pagenmbr)) != str:
        tokencirclist = tokencirclist + [tokencirdf(pagenmbr)]
        pagenmbr += 1

    tokenscirc = pd.concat(tokencirclist, axis = 0)
    tokens = pd.concat(tokenlist, axis = 0)
    tokens = tokens[tokens['MktCapUSD'] != '?']
    tokens = pd.concat([tokens.drop_duplicates(),tokenscirc.drop_duplicates()], axis = 1, join = 'inner')

    totaldf = pd.concat([coins,tokens], axis = 0)
    totaldf = totaldf[totaldf['CirculatingSupply'] != 'None']
    totaldf = totaldf[totaldf['Vol24USD'] != 'None']

    totaldf['MktCapUSD'] = pd.to_numeric(totaldf['MktCapUSD'])
    totaldf['CirculatingSupply'] = pd.to_numeric(totaldf['CirculatingSupply'])
    totaldf['TotalSupply'] = pd.to_numeric(totaldf['TotalSupply'])
    totaldf['Vol24USD'] = pd.to_numeric(totaldf['Vol24USD'])
    totaldf['VolRatio'] = totaldf['Vol24USD']/totaldf['MktCapUSD']
    totaldf['SupplyRatio'] = totaldf['CirculatingSupply']/totaldf['TotalSupply']
    
    totaldf = totaldf[totaldf['MktCapUSD'] <= mcapthreshold]
    totaldf = totaldf[totaldf['VolRatio'] >= volthreshold]
    totaldf = totaldf[totaldf['SupplyRatio'] >= supplythreshold]
    
    
    return totaldf

In [11]:
def checkannouncement(cmcname):

    site = 'https://coinmarketcap.com/currencies/' + cmcname + '/'

    res = requests.get(site)
    soup = str(bs(res.text, 'lxml'))

    if "Announcement" in soup:
        announcementsite = soup.split('<li><span class="glyphicon glyphicon-bullhorn text-gray" title="Announcement">')[1].split('<a href="')[1].split('" target')[0]
    else:
        announcementsite = "NA"

    if 'bitcointalk.org' in announcementsite:
        return 1, announcementsite
    else:
        return 0, 'NA'
    
def bctcheck(cmcname):

    annsite = checkannouncement(cmcname)[1]
    parsesoup = annsite.split('=')[0] + '=' + annsite.split('=')[1].split('.')[0]
    res = requests.get(parsesoup)
    soup = str(bs(res.text, 'lxml'))
    discussion = soup.split('<td class="middletext"')[1].split('<span class="prevnext">')[0]
    discussion = discussion.split('<div class="nav"')[0]
    if 'All' in discussion.split(parsesoup)[-1]:
        lastdiscuss = discussion.split(parsesoup)[-2]
    else:
        lastdiscuss = discussion.split(parsesoup)[-1]
        
    if '<td align="right" style="padding-right:' in discussion:
        lastpgIndex = lastdiscuss.split('">')[0]
    else:
        lastpgIndex = lastdiscuss.split('">')[0]

    lastpg = parsesoup + lastpgIndex
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    totpages = soup.split('<td class="middletext"')[1].split('[<b>')[1].split('</b>]')[0]

    forumparse = lastpg.split('=')[1]
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    lastmsg = soup.split(forumparse)[1].split('<tbody>')[-1].split('<div class="smalltext">')[-1].split('</div>')[0]
    
    return totpages, lastmsg

In [ ]:
'''
# Using old method to build dataframe, no change fields available

cmcdf = microfilter1(100000000, 0.02, 0.1)

names = list(cmcdf.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        totpgs, last = bctcheck(cmcname)
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcdf,bctdf],axis = 1)
filterdf.to_csv('cmc2.csv')

'''

In [47]:
cmcfull = pullfulldf()

names = list(cmcfull.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        try:
            totpgs, last = bctcheck(cmcname)
        except:
            totpgs, last = (0,0)
        
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcfull,bctdf],axis = 1)

binance = pd.read_csv('binancelist.csv')
binance.index = binance['Name']
del binance['Ticker'], binance['Name']

bittrex = pd.read_excel('bittrexlist.xlsx')
bittrex.index = bittrex['Name']
del bittrex['Ticker'], bittrex['Name']

kucoin = pd.read_csv('kucoinlist.csv')
kucoin.index = kucoin['Name']
del kucoin['Ticker'], kucoin['Name']

cryptopia = pd.read_csv('cryptopialist.csv')
cryptopia.index = cryptopia['Name']
del cryptopia['Ticker'], cryptopia['Name']

fulldf = pd.concat([filterdf,cryptopia,kucoin,binance,bittrex],axis = 1)
fulldf.head()

fulldf.to_csv('cmc.csv')

Number: 1 Coin: BTC
Number: 2 Coin: ETH
Number: 3 Coin: XRP
Number: 4 Coin: BCH
Number: 5 Coin: ADA
Number: 6 Coin: LTC
Number: 7 Coin: XEM
Number: 8 Coin: XLM
Number: 9 Coin: MIOTA
Number: 10 Coin: NEO
Number: 11 Coin: EOS
Number: 12 Coin: DASH
Number: 13 Coin: XMR
Number: 14 Coin: TRX
Number: 15 Coin: BTG
Number: 16 Coin: ETC
Number: 17 Coin: QTUM
Number: 18 Coin: ICX
Number: 19 Coin: LSK
Number: 20 Coin: XRB
Number: 21 Coin: OMG
Number: 22 Coin: XVG
Number: 23 Coin: BNB
Number: 24 Coin: ZEC
Number: 25 Coin: ARDR
Number: 26 Coin: BCN
Number: 27 Coin: SC
Number: 28 Coin: PPT
Number: 29 Coin: BCC
Number: 30 Coin: STRAT
Number: 31 Coin: VEN
Number: 32 Coin: KCS
Number: 33 Coin: SNT
Number: 34 Coin: USDT
Number: 35 Coin: BTS
Number: 36 Coin: STEEM
Number: 37 Coin: DOGE
Number: 38 Coin: DCN
Number: 39 Coin: SMART
Number: 40 Coin: WAVES
Number: 41 Coin: REP
Number: 42 Coin: ZRX
Number: 43 Coin: DRGN
Number: 44 Coin: DGB
Number: 45 Coin: VERI
Number: 46 Coin: KMD
Number: 47 Coin: ETN
Number

Number: 372 Coin: XAUR
Number: 373 Coin: GEO
Number: 374 Coin: BSD
Number: 375 Coin: ADC
Number: 376 Coin: BWK
Number: 377 Coin: PINK
Number: 378 Coin: WCT
Number: 379 Coin: HUSH
Number: 380 Coin: IOP
Number: 381 Coin: ABY
Number: 382 Coin: ECOB
Number: 383 Coin: BITUSD
Number: 384 Coin: TIME
Number: 385 Coin: WISH
Number: 386 Coin: SPF
Number: 387 Coin: ALQO
Number: 388 Coin: SUMO
Number: 389 Coin: XPM
Number: 390 Coin: ATMS
Number: 391 Coin: OXY
Number: 392 Coin: QAU
Number: 393 Coin: EAC
Number: 394 Coin: PIRL
Number: 395 Coin: ESP
Number: 396 Coin: XMCC
Number: 397 Coin: PLU
Number: 398 Coin: LEO
Number: 399 Coin: FRST
Number: 400 Coin: CSNO
Number: 401 Coin: CVCOIN
Number: 402 Coin: UNO
Number: 403 Coin: FYP
Number: 404 Coin: XVC
Number: 405 Coin: OBITS
Number: 406 Coin: SPR
Number: 407 Coin: SPHR
Number: 408 Coin: ZNY
Number: 409 Coin: BTM
Number: 410 Coin: SEND
Number: 411 Coin: MAG
Number: 412 Coin: DICE
Number: 413 Coin: NTRN
Number: 414 Coin: CURE
Number: 415 Coin: NVST
Numbe

Number: 737 Coin: PXC
Number: 738 Coin: BLAS
Number: 739 Coin: BUCKS
Number: 740 Coin: 8BIT
Number: 741 Coin: PR
Number: 742 Coin: CHESS
Number: 743 Coin: XHI
Number: 744 Coin: ONX
Number: 745 Coin: MARS
Number: 746 Coin: HNC
Number: 747 Coin: TGC
Number: 748 Coin: TSE
Number: 749 Coin: VIDZ
Number: 750 Coin: CHAN
Number: 751 Coin: XGR
Number: 752 Coin: MNM
Number: 753 Coin: MOJO
Number: 754 Coin: STS
Number: 755 Coin: ARI
Number: 756 Coin: NTO
Number: 757 Coin: MNC
Number: 758 Coin: XRA
Number: 759 Coin: XPY
Number: 760 Coin: LTB
Number: 761 Coin: BCF
Number: 762 Coin: UNY
Number: 763 Coin: PCOIN
Number: 764 Coin: UNIC
Number: 765 Coin: SLG
Number: 766 Coin: UNI
Number: 767 Coin: CCN
Number: 768 Coin: XCT
Number: 769 Coin: BRAT
Number: 770 Coin: 808
Number: 771 Coin: JIN
Number: 772 Coin: COAL
Number: 773 Coin: LCP
Number: 774 Coin: LBTC
Number: 775 Coin: YTN
Number: 776 Coin: CNO
Number: 777 Coin: TAG
Number: 778 Coin: BLC
Number: 779 Coin: 888
Number: 780 Coin: EL
Number: 781 Coin: 

Number: 1095 Coin: ATMC
Number: 1096 Coin: SWFTC
Number: 1097 Coin: SHOW
Number: 1098 Coin: BIX
Number: 1099 Coin: HPB
Number: 1100 Coin: WIC
Number: 1101 Coin: GNX
Number: 1102 Coin: AIDOC
Number: 1103 Coin: BCX
Number: 1104 Coin: MDS
Number: 1105 Coin: GTC
Number: 1106 Coin: SMT
Number: 1107 Coin: SBTC
Number: 1108 Coin: TIO
Number: 1109 Coin: QUN
Number: 1110 Coin: KCASH
Number: 1111 Coin: TOPC
Number: 1112 Coin: PRO
Number: 1113 Coin: TSL
Number: 1114 Coin: QLC
Number: 1115 Coin: FAIR
Number: 1116 Coin: HPY
Number: 1117 Coin: SPHTX
Number: 1118 Coin: CAN
Number: 1119 Coin: HTML
Number: 1120 Coin: VIU
Number: 1121 Coin: UGC
Number: 1122 Coin: MDT
Number: 1123 Coin: KBR
Number: 1124 Coin: QBT
Number: 1125 Coin: AIX
Number: 1126 Coin: CAT
Number: 1127 Coin: AMM
Number: 1128 Coin: XTZ
Number: 1129 Coin: MOT
Number: 1130 Coin: CAPP
Number: 1131 Coin: IGNIS
Number: 1132 Coin: B2X
Number: 1133 Coin: UBTC
Number: 1134 Coin: LCT
Number: 1135 Coin: BIG
Number: 1136 Coin: CYDER
Number: 1137 C

Name: 16 Alt: ethereum-classic
Name: 17 Alt: qtum
Name: 18 Alt: icon
Name: 19 Alt: lisk
Name: 20 Alt: raiblocks
Name: 21 Alt: omisego
Name: 22 Alt: verge
Name: 23 Alt: binance-coin
Name: 24 Alt: zcash
Name: 25 Alt: ardor
Name: 26 Alt: bytecoin-bcn
Name: 27 Alt: siacoin
Name: 28 Alt: populous
Name: 29 Alt: bitconnect
Name: 30 Alt: stratis
Name: 31 Alt: vechain
Name: 32 Alt: kucoin-shares
Name: 33 Alt: status
Name: 34 Alt: tether
Name: 35 Alt: bitshares
Name: 36 Alt: steem
Name: 37 Alt: dogecoin
Name: 38 Alt: dentacoin
Name: 39 Alt: smartcash
Name: 40 Alt: waves
Name: 41 Alt: augur
Name: 42 Alt: 0x
Name: 43 Alt: dragonchain
Name: 44 Alt: digibyte
Name: 45 Alt: veritaseum
Name: 46 Alt: komodo
Name: 47 Alt: electroneum
Name: 48 Alt: hshare
Name: 49 Alt: ark
Name: 50 Alt: qash
Name: 51 Alt: golem-network-tokens
Name: 52 Alt: loopring
Name: 53 Alt: wax
Name: 54 Alt: decred
Name: 55 Alt: walton
Name: 56 Alt: basic-attention-token
Name: 57 Alt: byteball
Name: 58 Alt: pivx
Name: 59 Alt: zclassi

Name: 360 Alt: luxcoin
Name: 361 Alt: icos
Name: 362 Alt: clams
Name: 363 Alt: primas
Name: 364 Alt: sequence
Name: 365 Alt: bitqy
Name: 366 Alt: circuits-of-value
Name: 367 Alt: heat-ledger
Name: 368 Alt: sprouts
Name: 369 Alt: vsync-vsx
Name: 370 Alt: solaris
Name: 371 Alt: aventus
Name: 372 Alt: xaurum
Name: 373 Alt: geocoin
Name: 374 Alt: bitsend
Name: 375 Alt: audiocoin
Name: 376 Alt: bulwark
Name: 377 Alt: pinkcoin
Name: 378 Alt: waves-community-token
Name: 379 Alt: hush
Name: 380 Alt: internet-of-people
Name: 381 Alt: artbyte
Name: 382 Alt: ecobit
Name: 383 Alt: bitusd
Name: 384 Alt: chronobank
Name: 385 Alt: mywish
Name: 386 Alt: sportyfi
Name: 387 Alt: alqo
Name: 388 Alt: sumokoin
Name: 389 Alt: primecoin
Name: 390 Alt: atmos
Name: 391 Alt: oxycoin
Name: 392 Alt: quantum
Name: 393 Alt: earthcoin
Name: 394 Alt: pirl
Name: 395 Alt: espers
Name: 396 Alt: monacocoin
Name: 397 Alt: pluton
Name: 398 Alt: leocoin
Name: 399 Alt: firstcoin
Name: 400 Alt: bitdice
Name: 401 Alt: cvcoin
N

Name: 693 Alt: 42-coin
Name: 694 Alt: chronos
Name: 695 Alt: britcoin
Name: 696 Alt: titcoin
Name: 697 Alt: crystal-clear
Name: 698 Alt: iethereum
Name: 699 Alt: sugar-exchange
Name: 700 Alt: bitcurrency
Name: 701 Alt: shadowcash
Name: 702 Alt: wavesgo
Name: 703 Alt: happycoin
Name: 704 Alt: zcash-gold
Name: 705 Alt: colossuscoin-v2
Name: 706 Alt: rimbit
Name: 707 Alt: bitstar
Name: 708 Alt: platinumbar
Name: 709 Alt: pascal-lite
Name: 710 Alt: amsterdamcoin
Name: 711 Alt: macron
Name: 712 Alt: cryptojacks
Name: 713 Alt: monkey-project
Name: 714 Alt: kobocoin
Name: 715 Alt: dollarcoin
Name: 716 Alt: funcoin
Name: 717 Alt: iticoin
Name: 718 Alt: dashcoin
Name: 719 Alt: supercoin
Name: 720 Alt: postoken
Name: 721 Alt: adcoin
Name: 722 Alt: kushcoin
Name: 723 Alt: triangles
Name: 724 Alt: dix-asset
Name: 725 Alt: postcoin
Name: 726 Alt: pakcoin
Name: 727 Alt: opal
Name: 728 Alt: giga-watt-token
Name: 729 Alt: kilocoin
Name: 730 Alt: whalecoin
Name: 731 Alt: signatum
Name: 732 Alt: goldblo

Name: 1018 Alt: mutual-coin
Name: 1019 Alt: mao-zedong
Name: 1020 Alt: ereal
Name: 1021 Alt: btctalkcoin
Name: 1022 Alt: bitsilver
Name: 1023 Alt: shilling
Name: 1024 Alt: joincoin
Name: 1025 Alt: firecoin
Name: 1026 Alt: freicoin
Name: 1027 Alt: cypher
Name: 1028 Alt: sproutsextreme
Name: 1029 Alt: islacoin
Name: 1030 Alt: newbium
Name: 1031 Alt: shadow-token
Name: 1032 Alt: sling
Name: 1033 Alt: sacoin
Name: 1034 Alt: cashcoin
Name: 1035 Alt: wmcoin
Name: 1036 Alt: franko
Name: 1037 Alt: yacoin
Name: 1038 Alt: coexistcoin
Name: 1039 Alt: virtualcoin
Name: 1040 Alt: satoshimadness
Name: 1041 Alt: ether-for-the-rest-of-the-world
Name: 1042 Alt: crypto
Name: 1043 Alt: piecoin
Name: 1044 Alt: secretcoin
Name: 1045 Alt: pulse
Name: 1046 Alt: guccionecoin
Name: 1047 Alt: vector
Name: 1048 Alt: jewels
Name: 1049 Alt: ripto-bux
Name: 1050 Alt: vip-tokens
Name: 1051 Alt: quebecoin
Name: 1052 Alt: bios-crypto
Name: 1053 Alt: warp
Name: 1054 Alt: flavorcoin
Name: 1055 Alt: cryptoworldx-token
Na

Name: 1334 Alt: blazercoin
Name: 1335 Alt: teracoin
Name: 1336 Alt: stex
Name: 1337 Alt: the-vegan-initiative
Name: 1338 Alt: dashs
Name: 1339 Alt: rupaya-old
Name: 1340 Alt: royalcoin
Name: 1341 Alt: safe-trade-coin
Name: 1342 Alt: happy-creator-coin
Name: 1343 Alt: quotient
Name: 1344 Alt: cheapcoin
Name: 1345 Alt: moneycoin
Name: 1346 Alt: avatarcoin
Name: 1347 Alt: pabyosi-coin-special
Name: 1348 Alt: eusd
Name: 1349 Alt: 10mtoken
Name: 1350 Alt: cycling-coin
Name: 1351 Alt: bubble
Name: 1352 Alt: aseancoin
Name: 1353 Alt: deuscoin
Name: 1354 Alt: faceblock
Name: 1355 Alt: xaucoin
Name: 1356 Alt: lepen
Name: 1357 Alt: first-bitcoin
Name: 1358 Alt: eltc
Name: 1359 Alt: x2
Name: 1360 Alt: digital-bullion-gold
Name: 1361 Alt: fireflycoin
Name: 1362 Alt: pinkdog
Name: 1363 Alt: kashhcoin
Name: 1364 Alt: cybercoin
Name: 1365 Alt: pokecoin
Name: 1366 Alt: wa-space
Name: 1367 Alt: teamup
Name: 1368 Alt: yescoin
Name: 1369 Alt: cbd-crystals
Name: 1370 Alt: moneta2
Name: 1371 Alt: topaz
Nam